![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

In [2]:
WITH top_industries AS (
    SELECT i.industry, 
        COUNT(*) AS industry_count
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    WHERE EXTRACT(year FROM d.date_joined) IN (2019, 2020, 2021)
    GROUP BY i.industry
    ORDER BY industry_count DESC
    LIMIT 3
),

yearly_rankings AS (
    SELECT i.industry,
        EXTRACT(year FROM d.date_joined) AS year,
        COUNT(*) AS num_unicorns,
        AVG(f.valuation) AS average_valuation
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    INNER JOIN funding AS f
        ON d.company_id = f.company_id
    GROUP BY i.industry, year
)

SELECT yr.industry,
    yr.year,
    yr.num_unicorns,
    ROUND(AVG(yr.average_valuation) / 1000000000, 2) AS average_valuation_billions
FROM yearly_rankings AS yr
WHERE yr.year IN (2019, 2020, 2021)
    AND yr.industry IN (SELECT industry FROM top_industries)
GROUP BY yr.industry, yr.year, yr.num_unicorns
ORDER BY yr.year DESC, yr.num_unicorns DESC;


,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58



**Top 3 industries**

The selected industries—Fintech, Internet Software & Services, and E-commerce & Direct-to-Consumer—are consistent across all years.

This confirms that the selection of top industries was correctly based on total unicorn count from 2019-2021.
Trends in Unicorn Creation

2021 saw the highest number of unicorns across all industries, indicating a strong startup growth trend.
Fintech led in 2021 (138 unicorns), followed by Internet Software & Services (119) and E-commerce (47).
The number of unicorns declined sharply in 2020 and 2019, as expected.
Average Valuation Trends

Valuations were highest in 2019, especially in Fintech ($6.8B) and Internet Software & Services ($4.23B).
2020 valuations remained high, particularly in Internet Software & Services ($4.35B) and Fintech ($4.33B).
2021 saw lower average valuations, suggesting a larger number of unicorns but possibly lower individual valuations.